In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

## Basic app with Pydantic Output Parser

In [3]:
from langchain.output_parsers import PydanticOutputParser

In [4]:
# from langchain_core.pydantic_v1 import BaseModel, Field, validator
from pydantic import BaseModel, Field, validator

In [5]:
from typing import List

### Define desired output data structure

In [6]:
# class Suggestions_Output_Structure(BaseModel):
#     words: List[str] = Field(
#         description="list of substitute words based on the context"
#     )
#     reasons: List[str] = Field(
#         description="the reasoning of why this word fits the context"
#     )

#     #Throw error if the substitute word starts with a number
#     @validator('words')
#     def not_start_with_number(cls, info):
#         for item in info:
#             if item[0].isnumeric():
#                 raise ValueError("ERROR: The word cannot start with a number")
#         return info

#     @validator('reasons')
#     def end_with_dot(cls, info):
#       for idx, item in enumerate(info):
#         if item[-1] != ".":
#           info[idx] += "."
#       return info

# [Pydantic V2: Migration Guide]
# - https://docs.pydantic.dev/2.10/migration/

from pydantic import BaseModel, Field, field_validator
from typing import List

class Suggestions_Output_Structure(BaseModel):
    words: List[str] = Field(
        description="list of substitute words based on the context"
    )
    reasons: List[str] = Field(
        description="the reasoning of why this word fits the context"
    )

    @field_validator('words', mode='before')
    @classmethod
    def not_start_with_number(cls, v):
        for item in v:
            if item[0].isdigit():
                raise ValueError("ERROR: The word cannot start with a number")
        return v

    @field_validator('reasons', mode='before')
    @classmethod
    def end_with_dot(cls, v):
        return [item if item.endswith('.') else item + '.' for item in v]


### Create parser

In [7]:
my_parser = PydanticOutputParser(
    pydantic_object=Suggestions_Output_Structure
)

### Determine input

In [8]:
from langchain.prompts import PromptTemplate

In [9]:
my_template = """
Offer a list of suggestions to substitute the specified
target_word based on the present context and the reasoning
for each word.

{format_instructions}

target_word={target_word}
context={context}
"""

In [10]:
my_prompt = PromptTemplate(
    template=my_template,
    input_variables=["target_word", "context"],
    partial_variables={
        "format_instructions": my_parser.get_format_instructions()
    }
)

In [11]:
user_input = my_prompt.format_prompt(
    target_word="loyalty",
    context="""
    The loyalty of the soldier was so great that
    even under severe torture, he refused to betray
    his comrades.
    """
)

In [12]:
# from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

In [13]:
# llm = OpenAI()
llm = ChatOpenAI(model=MODEL_GPT)

In [14]:
# output = llm(user_input.to_string())
output = llm.invoke(user_input.to_string())

### Apply parser to get desired output structure

In [15]:
print(output)

content='```json\n{\n  "words": ["fidelity", "devotion", "allegiance", "faithfulness", "commitment"],\n  "reasons": [\n    "Fidelity captures the idea of being faithful to a person or cause, similar to the soldier\'s unwavering support.",\n    "Devotion emphasizes a strong love or loyalty, reflecting the soldier\'s dedication to his comrades.",\n    "Allegiance refers to the loyalty owed to one\'s country or comrades, making it a fitting substitute in a military context.",\n    "Faithfulness suggests a steadfastness and reliability in one\'s commitments, aligning with the soldier\'s character.",\n    "Commitment embodies the idea of being dedicated to a cause or person, resonating with the soldier\'s resolve."\n  ]\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 279, 'total_tokens': 434, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_

In [16]:
print(output.content)

```json
{
  "words": ["fidelity", "devotion", "allegiance", "faithfulness", "commitment"],
  "reasons": [
    "Fidelity captures the idea of being faithful to a person or cause, similar to the soldier's unwavering support.",
    "Devotion emphasizes a strong love or loyalty, reflecting the soldier's dedication to his comrades.",
    "Allegiance refers to the loyalty owed to one's country or comrades, making it a fitting substitute in a military context.",
    "Faithfulness suggests a steadfastness and reliability in one's commitments, aligning with the soldier's character.",
    "Commitment embodies the idea of being dedicated to a cause or person, resonating with the soldier's resolve."
  ]
}
```


In [17]:
# my_parser.parse(output)
my_parser.parse(output.content)

Suggestions_Output_Structure(words=['fidelity', 'devotion', 'allegiance', 'faithfulness', 'commitment'], reasons=["Fidelity captures the idea of being faithful to a person or cause, similar to the soldier's unwavering support.", "Devotion emphasizes a strong love or loyalty, reflecting the soldier's dedication to his comrades.", "Allegiance refers to the loyalty owed to one's country or comrades, making it a fitting substitute in a military context.", "Faithfulness suggests a steadfastness and reliability in one's commitments, aligning with the soldier's character.", "Commitment embodies the idea of being dedicated to a cause or person, resonating with the soldier's resolve."])